In [1]:
cd ../task_01/pipelines

/Users/pldelacour/Documents/PL_Ecole/data_sc_lab/data_sc_lab/task_01/pipelines


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipynb.fs.full.tool_functions import *
from yellowbrick.regressor import AlphaSelection
from sklearn.model_selection import train_test_split
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [3]:
df_beha = pd.read_csv('../../../data/Behavioral/AllData.csv')#, nrows=1000)

df_spectro_cluster = pd.read_csv("../../../data/EEG/RestingEEG_Spectro_Cluster.csv") #, nrows=1000)

srs = pd.read_csv('../../../data/HBN_srs.csv', delimiter=';')
outliers = pd.read_csv('../../../data/outliers.csv')

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,11

In [4]:
#s_in = df_beha[['SWAN_IN_Avg' , 'Age', 'Patient_ID']]
#s_HY = df_beha[['SWAN_HY_Avg' , 'Age', 'Patient_ID']]

In [5]:
#df_beha[['Sex']]

## SWAN

In [6]:
swan = df_beha[['SWAN_IN_Avg' , 'SWAN_HY_Avg', 'Patient_ID']]
in_null = swan['SWAN_IN_Avg'].isnull().sum()
hy_null = swan['SWAN_HY_Avg'].isnull().sum()
#print(in_null)
#print(hy_null)
swan = swan[~ (swan['SWAN_IN_Avg'].isnull() | swan['SWAN_HY_Avg'].isnull()) ]

## SRS

In [7]:
srs = srs[~srs['SRS_RRB'].isnull()]
srs.rename(columns={'IDs': 'Patient_ID'}, inplace=True)

In [8]:
scores = pd.merge(swan , srs , on = 'Patient_ID')
scores

,SWAN_IN_Avg,SWAN_HY_Avg,Patient_ID,Sex,Age,SRS_RRB,SRS_SCI,SRS_Total
0,0.000000,1.222222,NDARYM832PX3,1,7.048254,10.0,45.0,55.0
1,1.222222,0.555555,NDARNJ687DMC,1,6.348163,19.0,73.0,92.0
2,1.111111,1.888888,NDARED632KNG,1,17.203855,2.0,28.0,30.0
3,-0.111111,2.222222,NDARFM080VAF,1,7.623203,14.0,72.0,86.0
4,-0.111111,0.777777,NDARFW130NGG,1,8.316107,25.0,85.0,110.0
...,...,...,...,...,...,...,...,...
1234,-1.444444,-2.333333,NDARND697FLK,1,6.517796,1.0,17.0,18.0
1235,1.222222,0.444444,NDARCL008HLA,0,16.456194,21.0,82.0,103.0
1236,-1.888888,-3.000000,NDARYM334BZ5,0,16.119438,0.0,6.0,6.0
1237,-0.666666,1.333333,NDARYZ986HEW,1,7.563084,9.0,49.0,58.0


## Combine with spectro cluster 

In [9]:
df_spectro_cluster.rename(columns={'id': 'Patient_ID'}, inplace=True)
df_label_spectro_cluster = pd.merge(scores,  df_spectro_cluster, on='Patient_ID')

df_label_spectro_cluster = fill_with_median(df_label_spectro_cluster)

In [10]:
less_18 = df_label_spectro_cluster['Age'] <= 18.0
df_label_spectro_cluster = df_label_spectro_cluster[less_18]

In [11]:
df_label_spectro_cluster

,SWAN_IN_Avg,SWAN_HY_Avg,Patient_ID,Sex,Age,SRS_RRB,SRS_SCI,SRS_Total,eyesclosed_fband_delta_absmean_lfront,eyesclosed_fband_delta_absmean_mfront,...,eyesopen_fooof_peak_freq_rfront,eyesopen_fooof_peak_freq_lpari,eyesopen_fooof_peak_freq_mpari,eyesopen_fooof_peak_freq_rpari,eyesopen_fooof_peak_amplitude_lfront,eyesopen_fooof_peak_amplitude_mfront,eyesopen_fooof_peak_amplitude_rfront,eyesopen_fooof_peak_amplitude_lpari,eyesopen_fooof_peak_amplitude_mpari,eyesopen_fooof_peak_amplitude_rpari
0,-0.111111,2.222222,NDARFM080VAF,1,7.623203,14.0,72.0,86.0,7.519803,7.043891,...,9.485781,9.441022,9.384172,9.425798,0.795549,0.876262,0.872379,0.989101,1.099981,0.915441
1,-0.444444,0.444444,NDARVG148WHV,0,10.835158,17.0,70.0,87.0,2.346685,3.945786,...,8.800915,8.510134,9.078390,14.741810,0.254701,0.351010,0.303875,0.234960,0.232564,0.344537
2,1.333333,1.444444,NDARUY876VL1,0,7.987451,19.0,57.0,76.0,1.281869,1.229748,...,5.604535,5.602136,4.096616,5.084040,0.048655,0.233294,0.282735,0.256238,0.175123,0.304286
3,0.666666,-0.888888,NDARYU323ZDJ,1,8.096851,12.0,58.0,70.0,3.109075,4.803264,...,9.288451,6.651558,6.396867,10.291086,0.218827,0.313734,0.298905,0.178899,0.108880,0.449437
4,0.111111,1.222222,NDARCP360AFD,0,8.357061,15.0,68.0,83.0,5.786769,6.818487,...,6.952035,5.065536,4.679294,4.924409,0.150518,0.169058,0.154417,0.104574,0.102992,0.094126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,-1.444444,-2.333333,NDARND697FLK,1,6.517796,1.0,17.0,18.0,8.594610,13.258798,...,5.868008,22.846455,7.050818,4.622321,0.416442,0.358625,0.380528,0.169463,0.325197,0.199935
902,1.222222,0.444444,NDARCL008HLA,0,16.456194,21.0,82.0,103.0,7.487871,9.809903,...,9.389994,9.540163,9.482167,9.545163,1.326583,1.288238,1.407225,1.561878,1.636154,1.412149
903,-1.888888,-3.000000,NDARYM334BZ5,0,16.119438,0.0,6.0,6.0,3.743134,3.795633,...,10.376962,10.707490,10.383040,10.343315,0.623514,0.582917,0.422394,0.615360,0.860283,0.945789
904,-0.666666,1.333333,NDARYZ986HEW,1,7.563084,9.0,49.0,58.0,2.830657,5.259391,...,9.482208,7.101976,9.409898,9.301044,0.185029,0.165101,0.246363,0.120704,0.254212,0.271848


In [12]:
# Remove preidentified outliers

In [13]:
outliers = pd.read_csv('../../../data/outliers.csv')
df_label_spectro_cluster = df_label_spectro_cluster[~df_label_spectro_cluster.Patient_ID.isin(outliers.Patient_ID)]

## Train Test split 

In [14]:
X = df_label_spectro_cluster.drop([ 'SWAN_IN_Avg' , 'SWAN_HY_Avg', 'SRS_Total' , 'SRS_RRB' , 'SRS_SCI', 'Patient_ID'], axis=1)
Y = df_label_spectro_cluster[[ 'SWAN_IN_Avg' , 'SWAN_HY_Avg', 'SRS_Total' , 'SRS_RRB' , 'SRS_SCI']]


In [15]:
X

,Sex,Age,eyesclosed_fband_delta_absmean_lfront,eyesclosed_fband_delta_absmean_mfront,eyesclosed_fband_delta_absmean_rfront,eyesclosed_fband_delta_absmean_lpari,eyesclosed_fband_delta_absmean_mpari,eyesclosed_fband_delta_absmean_rpari,eyesclosed_fband_delta_relmean_lfront,eyesclosed_fband_delta_relmean_mfront,...,eyesopen_fooof_peak_freq_rfront,eyesopen_fooof_peak_freq_lpari,eyesopen_fooof_peak_freq_mpari,eyesopen_fooof_peak_freq_rpari,eyesopen_fooof_peak_amplitude_lfront,eyesopen_fooof_peak_amplitude_mfront,eyesopen_fooof_peak_amplitude_rfront,eyesopen_fooof_peak_amplitude_lpari,eyesopen_fooof_peak_amplitude_mpari,eyesopen_fooof_peak_amplitude_rpari
0,1,7.623203,7.519803,7.043891,6.528158,9.633529,9.989483,9.305804,9.598653,10.164125,...,9.485781,9.441022,9.384172,9.425798,0.795549,0.876262,0.872379,0.989101,1.099981,0.915441
1,0,10.835158,2.346685,3.945786,2.962716,3.440606,3.565384,3.023286,10.968235,12.461058,...,8.800915,8.510134,9.078390,14.741810,0.254701,0.351010,0.303875,0.234960,0.232564,0.344537
2,0,7.987451,1.281869,1.229748,1.905450,1.768091,1.484286,0.756394,13.132905,11.998897,...,5.604535,5.602136,4.096616,5.084040,0.048655,0.233294,0.282735,0.256238,0.175123,0.304286
3,1,8.096851,3.109075,4.803264,2.873502,2.644382,3.884837,2.391569,9.931212,10.869944,...,9.288451,6.651558,6.396867,10.291086,0.218827,0.313734,0.298905,0.178899,0.108880,0.449437
4,0,8.357061,5.786769,6.818487,6.296279,8.043145,9.413683,6.839451,10.495082,11.095547,...,6.952035,5.065536,4.679294,4.924409,0.150518,0.169058,0.154417,0.104574,0.102992,0.094126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,1,6.517796,8.594610,13.258798,7.158682,12.743212,8.698643,4.855232,15.315470,14.779320,...,5.868008,22.846455,7.050818,4.622321,0.416442,0.358625,0.380528,0.169463,0.325197,0.199935
902,0,16.456194,7.487871,9.809903,7.255467,8.731814,9.629680,11.326718,7.404931,7.060421,...,9.389994,9.540163,9.482167,9.545163,1.326583,1.288238,1.407225,1.561878,1.636154,1.412149
903,0,16.119438,3.743134,3.795633,3.286381,2.779438,3.539956,3.800533,8.565498,8.383859,...,10.376962,10.707490,10.383040,10.343315,0.623514,0.582917,0.422394,0.615360,0.860283,0.945789
904,1,7.563084,2.830657,5.259391,6.724664,4.745513,5.885628,3.160138,13.388712,13.121943,...,9.482208,7.101976,9.409898,9.301044,0.185029,0.165101,0.246363,0.120704,0.254212,0.271848


In [16]:

x_train , x_test, y_train , y_test = train_test_split(X,Y, test_size=0.2, random_state = 0)


In [17]:
forest = IsolationForest()
forest.fit(x_train)

# Outlier indices for training
outliers_training = forest.predict(x_train)
outliers_training_indices = np.argwhere(outliers_training == 1).flatten()

# Drop signal outliers in training data
x_train = x_train[outliers_training == 1]
y_train = y_train[outliers_training == 1]
#x_train, y_train = drop_outliers_samples_isolation_forest(x_train, y_train)
print(outliers_training_indices)
print(x_test.shape)
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  17  18
  19  20  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37  38
  39  40  41  42  43  44  45  47  49  50  51  52  53  54  55  57  58  59
  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77
  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95
  97  98  99 100 101 102 103 104 105 106 107 108 109 110 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 181 182 183 184 185 186 187 188 189 190
 191 192 193 194 195 196 197 198 199 200 201 203 204 205 207 208 209 210
 211 212 213 214 215 216 219 220 221 222 223 224 225 226 227 229 230 231
 232 233 234 235 236 237 238 239 240 241 242 243 244 245 247 248 249 250
 251 252 253 254 255 256 257 258 259 260 261 262 26

In [18]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = pd.DataFrame(scaler.transform(x_train), columns=x_train.columns, index=x_train.index)
x_test = pd.DataFrame(scaler.transform(x_test), columns=x_test.columns, index=x_test.index)


## Model

# Scaling the y

In [19]:
def normalize(y) : 
    y_mean = np.mean(y , axis = 0)
    y_std = np.std(y , axis = 0)
    y_scaled = (y - y_mean)/y_std
    return (y_scaled , y_mean , y_std )
def normalize_p(y , m , s):
    return (y - m)/s
def denormalize( y_scaled ,y_mean , y_std ):
    y = y_scaled*y_std +y_mean
    return y

In [20]:
y_train_scaled , y_mean, y_std ,  = normalize(y_train)


In [21]:
# get the dataset
 
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(50, input_dim=n_inputs, activation='relu'))
    model.add(Dense(50, input_dim=n_inputs, activation='relu'))
    #model.add(Dense(50, input_dim=n_inputs, activation='relu'))
    #model.add(Dense(50, input_dim=n_inputs, activation='relu'))
    #model.add(Dense(50, input_dim=n_inputs, activation='relu'))
    model.add(Dense(n_outputs, activation=None))
    model.compile(loss='mse' , optimizer='adam')
    return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
    results = list()
    n_inputs, n_outputs = X.shape[1], y.shape[1]
    
    # define evaluation procedure
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # enumerate folds
    for train_ix, test_ix in cv.split(X):
        # prepare data
        X_train, X_test = X[train_ix], X[test_ix]
        y_train, y_test = y[train_ix], y[test_ix]
        # define model
        model = get_model(n_inputs, n_outputs)
        # fit model
        model.fit(X_train, y_train, verbose=0, epochs=100)
        # evaluate model on test set
        mae = model.evaluate(X_test, y_test, verbose=0)
        # store result
        print('>%.3f' % mae)
    results.append(mae)
    return results


In [22]:
#results = evaluate_model(x_train.to_numpy(), y_train.to_numpy())
# summarize performance
#print('MAE: %.3f (%.3f)' % (mean(results), std(results)))

In [345]:
checkpointer = ModelCheckpoint(filepath='Best_model.h5', monitor='val_mse', verbose=1, save_best_only=True)
model = get_model(x_train.shape[1] , y_train.shape[1])
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='mse', verbose=1, save_best_only=True, mode='max')

###
early = EarlyStopping(monitor="mse", mode="max", patience=10, verbose=1)
redonplat = ReduceLROnPlateau(monitor="mse", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
###
hist = model.fit(
    x_train.to_numpy(), 
    y_train_scaled.to_numpy(), 
    validation_split = 1/6,
    batch_size=275, epochs=400, verbose=2, shuffle=True, 
    callbacks=[checkpointer])

Train on 560 samples, validate on 113 samples
Epoch 1/400
 - 0s - loss: 2.4061 - mse: 2.4061 - val_loss: 1.7578 - val_mse: 1.7578

Epoch 00001: val_mse improved from inf to 1.75783, saving model to Best_model.h5
Epoch 2/400
 - 0s - loss: 1.7092 - mse: 1.7092 - val_loss: 1.4941 - val_mse: 1.4941

Epoch 00002: val_mse improved from 1.75783 to 1.49415, saving model to Best_model.h5
Epoch 3/400
 - 0s - loss: 1.5233 - mse: 1.5233 - val_loss: 1.4005 - val_mse: 1.4005

Epoch 00003: val_mse improved from 1.49415 to 1.40048, saving model to Best_model.h5
Epoch 4/400
 - 0s - loss: 1.4145 - mse: 1.4145 - val_loss: 1.3318 - val_mse: 1.3318

Epoch 00004: val_mse improved from 1.40048 to 1.33183, saving model to Best_model.h5
Epoch 5/400
 - 0s - loss: 1.3308 - mse: 1.3308 - val_loss: 1.3312 - val_mse: 1.3312

Epoch 00005: val_mse improved from 1.33183 to 1.33123, saving model to Best_model.h5
Epoch 6/400
 - 0s - loss: 1.2930 - mse: 1.2930 - val_loss: 1.3791 - val_mse: 1.3791

Epoch 00006: val_mse di


Epoch 00056: val_mse did not improve from 1.03596
Epoch 57/400
 - 0s - loss: 0.7948 - mse: 0.7948 - val_loss: 1.0837 - val_mse: 1.0837

Epoch 00057: val_mse did not improve from 1.03596
Epoch 58/400
 - 0s - loss: 0.7799 - mse: 0.7799 - val_loss: 1.0860 - val_mse: 1.0860

Epoch 00058: val_mse did not improve from 1.03596
Epoch 59/400
 - 0s - loss: 0.7758 - mse: 0.7758 - val_loss: 1.0810 - val_mse: 1.0810

Epoch 00059: val_mse did not improve from 1.03596
Epoch 60/400
 - 0s - loss: 0.7720 - mse: 0.7720 - val_loss: 1.0673 - val_mse: 1.0673

Epoch 00060: val_mse did not improve from 1.03596
Epoch 61/400
 - 0s - loss: 0.7642 - mse: 0.7642 - val_loss: 1.0527 - val_mse: 1.0527

Epoch 00061: val_mse did not improve from 1.03596
Epoch 62/400
 - 0s - loss: 0.7640 - mse: 0.7640 - val_loss: 1.0555 - val_mse: 1.0555

Epoch 00062: val_mse did not improve from 1.03596
Epoch 63/400
 - 0s - loss: 0.7555 - mse: 0.7555 - val_loss: 1.1105 - val_mse: 1.1105

Epoch 00063: val_mse did not improve from 1.035

 - 0s - loss: 0.6016 - mse: 0.6016 - val_loss: 1.1072 - val_mse: 1.1072

Epoch 00117: val_mse did not improve from 1.03596
Epoch 118/400
 - 0s - loss: 0.5928 - mse: 0.5928 - val_loss: 1.1042 - val_mse: 1.1042

Epoch 00118: val_mse did not improve from 1.03596
Epoch 119/400
 - 0s - loss: 0.5822 - mse: 0.5822 - val_loss: 1.0942 - val_mse: 1.0942

Epoch 00119: val_mse did not improve from 1.03596
Epoch 120/400
 - 0s - loss: 0.5825 - mse: 0.5825 - val_loss: 1.0906 - val_mse: 1.0906

Epoch 00120: val_mse did not improve from 1.03596
Epoch 121/400
 - 0s - loss: 0.5795 - mse: 0.5795 - val_loss: 1.1031 - val_mse: 1.1031

Epoch 00121: val_mse did not improve from 1.03596
Epoch 122/400
 - 0s - loss: 0.5746 - mse: 0.5746 - val_loss: 1.1301 - val_mse: 1.1301

Epoch 00122: val_mse did not improve from 1.03596
Epoch 123/400
 - 0s - loss: 0.5828 - mse: 0.5828 - val_loss: 1.1283 - val_mse: 1.1283

Epoch 00123: val_mse did not improve from 1.03596
Epoch 124/400
 - 0s - loss: 0.5864 - mse: 0.5864 - val_

 - 0s - loss: 0.4592 - mse: 0.4592 - val_loss: 1.1925 - val_mse: 1.1925

Epoch 00177: val_mse did not improve from 1.03596
Epoch 178/400
 - 0s - loss: 0.4493 - mse: 0.4493 - val_loss: 1.1569 - val_mse: 1.1569

Epoch 00178: val_mse did not improve from 1.03596
Epoch 179/400
 - 0s - loss: 0.4447 - mse: 0.4447 - val_loss: 1.1487 - val_mse: 1.1487

Epoch 00179: val_mse did not improve from 1.03596
Epoch 180/400
 - 0s - loss: 0.4479 - mse: 0.4479 - val_loss: 1.1555 - val_mse: 1.1555

Epoch 00180: val_mse did not improve from 1.03596
Epoch 181/400
 - 0s - loss: 0.4566 - mse: 0.4566 - val_loss: 1.1918 - val_mse: 1.1918

Epoch 00181: val_mse did not improve from 1.03596
Epoch 182/400
 - 0s - loss: 0.4542 - mse: 0.4542 - val_loss: 1.2259 - val_mse: 1.2259

Epoch 00182: val_mse did not improve from 1.03596
Epoch 183/400
 - 0s - loss: 0.4432 - mse: 0.4432 - val_loss: 1.2486 - val_mse: 1.2486

Epoch 00183: val_mse did not improve from 1.03596
Epoch 184/400
 - 0s - loss: 0.4516 - mse: 0.4516 - val_

Epoch 237/400
 - 0s - loss: 0.3668 - mse: 0.3668 - val_loss: 1.3119 - val_mse: 1.3119

Epoch 00237: val_mse did not improve from 1.03596
Epoch 238/400
 - 0s - loss: 0.3701 - mse: 0.3701 - val_loss: 1.2469 - val_mse: 1.2469

Epoch 00238: val_mse did not improve from 1.03596
Epoch 239/400
 - 0s - loss: 0.3523 - mse: 0.3523 - val_loss: 1.2414 - val_mse: 1.2414

Epoch 00239: val_mse did not improve from 1.03596
Epoch 240/400
 - 0s - loss: 0.3650 - mse: 0.3650 - val_loss: 1.2701 - val_mse: 1.2701

Epoch 00240: val_mse did not improve from 1.03596
Epoch 241/400
 - 0s - loss: 0.3654 - mse: 0.3654 - val_loss: 1.2891 - val_mse: 1.2891

Epoch 00241: val_mse did not improve from 1.03596
Epoch 242/400
 - 0s - loss: 0.3506 - mse: 0.3506 - val_loss: 1.2706 - val_mse: 1.2706

Epoch 00242: val_mse did not improve from 1.03596
Epoch 243/400
 - 0s - loss: 0.3510 - mse: 0.3510 - val_loss: 1.2715 - val_mse: 1.2715

Epoch 00243: val_mse did not improve from 1.03596
Epoch 244/400
 - 0s - loss: 0.3502 - mse:

 - 0s - loss: 0.2759 - mse: 0.2759 - val_loss: 1.3298 - val_mse: 1.3298

Epoch 00297: val_mse did not improve from 1.03596
Epoch 298/400
 - 0s - loss: 0.2564 - mse: 0.2564 - val_loss: 1.3375 - val_mse: 1.3375

Epoch 00298: val_mse did not improve from 1.03596
Epoch 299/400
 - 0s - loss: 0.2705 - mse: 0.2705 - val_loss: 1.3653 - val_mse: 1.3653

Epoch 00299: val_mse did not improve from 1.03596
Epoch 300/400
 - 0s - loss: 0.2621 - mse: 0.2621 - val_loss: 1.4044 - val_mse: 1.4044

Epoch 00300: val_mse did not improve from 1.03596
Epoch 301/400
 - 0s - loss: 0.2638 - mse: 0.2638 - val_loss: 1.3854 - val_mse: 1.3854

Epoch 00301: val_mse did not improve from 1.03596
Epoch 302/400
 - 0s - loss: 0.2566 - mse: 0.2566 - val_loss: 1.3402 - val_mse: 1.3402

Epoch 00302: val_mse did not improve from 1.03596
Epoch 303/400
 - 0s - loss: 0.2764 - mse: 0.2764 - val_loss: 1.3498 - val_mse: 1.3498

Epoch 00303: val_mse did not improve from 1.03596
Epoch 304/400
 - 0s - loss: 0.2562 - mse: 0.2562 - val_

 - 0s - loss: 0.2338 - mse: 0.2338 - val_loss: 1.4604 - val_mse: 1.4604

Epoch 00357: val_mse did not improve from 1.03596
Epoch 358/400
 - 0s - loss: 0.2225 - mse: 0.2225 - val_loss: 1.4368 - val_mse: 1.4368

Epoch 00358: val_mse did not improve from 1.03596
Epoch 359/400
 - 0s - loss: 0.2100 - mse: 0.2100 - val_loss: 1.4107 - val_mse: 1.4107

Epoch 00359: val_mse did not improve from 1.03596
Epoch 360/400
 - 0s - loss: 0.2065 - mse: 0.2065 - val_loss: 1.4182 - val_mse: 1.4182

Epoch 00360: val_mse did not improve from 1.03596
Epoch 361/400
 - 0s - loss: 0.2070 - mse: 0.2070 - val_loss: 1.4418 - val_mse: 1.4418

Epoch 00361: val_mse did not improve from 1.03596
Epoch 362/400
 - 0s - loss: 0.2147 - mse: 0.2147 - val_loss: 1.4104 - val_mse: 1.4104

Epoch 00362: val_mse did not improve from 1.03596
Epoch 363/400
 - 0s - loss: 0.2034 - mse: 0.2034 - val_loss: 1.4179 - val_mse: 1.4179

Epoch 00363: val_mse did not improve from 1.03596
Epoch 364/400
 - 0s - loss: 0.2001 - mse: 0.2001 - val_


$\sum\limits_{i = 1}^n \| y_i -  \hat{y_i}\|^2$

In [346]:
y_pred = model.predict(x_test.to_numpy())
y_test_n = normalize_p(y_test,y_mean , y_std)

## Mean square error 

In [347]:
from sklearn.metrics import mean_squared_error

In [348]:
#y_pred = denormalize(y_pred , np.array(y_mean) , np.array(y_std))
for i in range(len(y_test.columns)):
    print(y_test.columns[i] , ':', mean_squared_error( y_test_n.to_numpy()[:,i] , y_pred[:,i]) )

SWAN_IN_Avg : 1.9966317777124991
SWAN_HY_Avg : 1.7786173711882058
SRS_Total : 1.673988663716875
SRS_RRB : 1.4446659676323625
SRS_SCI : 1.597689067197557


# Mean Baseline

In [349]:
y_baseline = np.zeros(y_test_n.shape) + np.mean(y_train_scaled.to_numpy() , axis=0)

for i in range(len(y_test.columns)):
    print(y_test.columns[i] , ':', mean_squared_error( y_test_n.to_numpy()[:,i] , y_baseline[:,i]) )

SWAN_IN_Avg : 1.0000256644070629
SWAN_HY_Avg : 0.9728785044418361
SRS_Total : 0.9163086905500325
SRS_RRB : 0.8761106624325128
SRS_SCI : 0.9179364884401283


## Using the Latent Space 

In [23]:
pwd

'/Users/pldelacour/Documents/PL_Ecole/data_sc_lab/data_sc_lab/task_01/pipelines'

In [24]:
df_x_train = pd.read_csv("../../../data/x_train_latent.csv")
df_x_test = pd.read_csv("../../../data/x_test_latent.csv")
df_beha = pd.read_csv('../../../data/Behavioral/AllData.csv')
srs = pd.read_csv('../../../data/HBN_srs.csv', delimiter=';')
outliers = pd.read_csv('../../../data/outliers.csv')

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (66,68,80,81,82,83,84,95,96,97,98,99,110,111,112,113,114,125,126,127,128,129,140,141,142,144,155,156,157,159,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053,1054,1055,1056,1057,1058,1059,1060,1061,1062,1063,1064,1065,1066,1067,1068,1069,1070,1071,1072,1073,1074,1075,1076,1077,1078,1079,1080,1081,1082,1083,1084,1085,1086,1087,1088,1089,1090,1091,1092,1093,1094,1095,1096,1097,1098,1099,11

In [25]:
df_x_train = df_x_train[~df_x_train.Patient_ID.isin(outliers.Patient_ID)]
df_x_test = df_x_test[~df_x_test.Patient_ID.isin(outliers.Patient_ID)]

## Swan

In [26]:
swan = df_beha[['SWAN_IN_Avg' , 'SWAN_HY_Avg',  'Patient_ID']]
in_null = swan['SWAN_IN_Avg'].isnull().sum()
hy_null = swan['SWAN_HY_Avg'].isnull().sum()
#print(in_null)
#print(hy_null)
swan = swan[~ (swan['SWAN_IN_Avg'].isnull() | swan['SWAN_HY_Avg'].isnull()) ]

# Residuals

In [27]:
age_ = df_beha[['Age', 'Patient_ID', "DX_01_Cat"]]
age_healthy = age_[age_["DX_01_Cat"].str.contains("No Diagnosis Given") == True]
age_healthy = age_healthy[['Age', 'Patient_ID']]

In [28]:
df_x_train_age_h = pd.merge(age_healthy, df_x_train, on='Patient_ID')
df_x_test_age_h = pd.merge(age_healthy, df_x_test, on='Patient_ID')

In [29]:
x_train_age_h, y_train_age_h = df_x_train_age_h.drop(['Unnamed: 0', 'Patient_ID', 'Age'], axis=1).to_numpy(), df_x_train_age_h[['Age']].to_numpy()
x_test_age_h, y_test_age_h = df_x_test_age_h.drop(['Unnamed: 0', 'Patient_ID', 'Age'], axis=1).to_numpy(), df_x_test_age_h[['Age']].to_numpy()

In [34]:
rdf = fit_random_forest_with_grid_search(x_train_latent[:100, 2:], x_train_latent[:100, 1])

/Users/pldelacour/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


Best parameters for Random Forest: {'max_depth': 25, 'max_leaf_nodes': 150, 'min_samples_split': 2, 'n_estimators': 1000}


In [35]:
rdf.best_score_

-5.895145784890754

In [36]:
import sklearn
sklearn.metrics.mean_squared_error(rdf.predict(x_train_latent[100:, 2:]), x_train_latent[100:, 1])


6.086340942874797

In [37]:
y_pred_train = rdf.predict(x_train_latent[100:, 2:])
y_pred_test = rdf.predict(x_test_latent[:,2:])

In [38]:
res_train = x_train_latent[100:, 1].reshape((-1, 1)) - y_pred_train.reshape((-1, 1))
res_test = x_test_latent[:, 1].reshape((-1, 1)) - y_pred_test.reshape((-1, 1))

In [39]:
res_train

array([[-1.65692084e-02],
       [ 9.24082734e-01],
       [ 1.66483970e+00],
       [-1.65198327e+00],
       [ 7.95931303e-01],
       [-2.32669131e+00],
       [ 1.51089662e+00],
       [-2.61371259e+00],
       [-3.87675082e-01],
       [ 8.83491737e-01],
       [-1.74009458e+00],
       [ 5.76524501e+00],
       [ 2.06210094e+00],
       [-1.19359824e+00],
       [-1.29981548e+00],
       [-6.68354716e-01],
       [-1.97433220e+00],
       [-1.23283675e+00],
       [ 1.24497486e+00],
       [-2.40678477e+00],
       [ 5.56050279e-01],
       [-7.19890456e-01],
       [-1.19246400e+00],
       [-3.70494784e+00],
       [-2.07084337e+00],
       [-2.66495136e+00],
       [-1.78917321e+00],
       [-5.94244696e-01],
       [-2.76157353e+00],
       [-1.33323172e+00],
       [-1.02176795e+00],
       [-7.75621411e-01],
       [ 2.33651865e+00],
       [-2.40757409e+00],
       [ 4.86846673e+00],
       [-1.14052835e-01],
       [-1.89800959e+00],
       [ 2.01365790e+00],
       [ 1.5

## SRS

In [40]:
srs = srs[~srs['SRS_RRB'].isnull()]

In [41]:
srs.rename(columns={'IDs': 'Patient_ID'}, inplace=True)

In [42]:
scores = pd.merge(swan , srs , on = 'Patient_ID')
scores

,SWAN_IN_Avg,SWAN_HY_Avg,Patient_ID,Sex,Age,SRS_RRB,SRS_SCI,SRS_Total
0,0.000000,1.222222,NDARYM832PX3,1,7.048254,10.0,45.0,55.0
1,1.222222,0.555555,NDARNJ687DMC,1,6.348163,19.0,73.0,92.0
2,1.111111,1.888888,NDARED632KNG,1,17.203855,2.0,28.0,30.0
3,-0.111111,2.222222,NDARFM080VAF,1,7.623203,14.0,72.0,86.0
4,-0.111111,0.777777,NDARFW130NGG,1,8.316107,25.0,85.0,110.0
...,...,...,...,...,...,...,...,...
1234,-1.444444,-2.333333,NDARND697FLK,1,6.517796,1.0,17.0,18.0
1235,1.222222,0.444444,NDARCL008HLA,0,16.456194,21.0,82.0,103.0
1236,-1.888888,-3.000000,NDARYM334BZ5,0,16.119438,0.0,6.0,6.0
1237,-0.666666,1.333333,NDARYZ986HEW,1,7.563084,9.0,49.0,58.0


In [43]:
df_x_train_srs = pd.merge(scores, df_x_train, on='Patient_ID')
df_x_test_srs = pd.merge(scores, df_x_test, on='Patient_ID')

In [44]:
x_train_latent = df_x_train_srs.drop(['SWAN_IN_Avg' ,'SWAN_HY_Avg' , 'SRS_RRB' , 'SRS_SCI', 'SRS_Total', 'Patient_ID', 'Unnamed: 0'], axis = 1).to_numpy()
y_train_latent= df_x_train_srs[['SWAN_IN_Avg' ,'SWAN_HY_Avg' ,'SRS_RRB' , 'SRS_SCI']]
x_test_latent = df_x_test_srs.drop(['SWAN_IN_Avg' ,'SWAN_HY_Avg' , 'SRS_RRB' , 'SRS_SCI', 'SRS_Total', 'Patient_ID', 'Unnamed: 0'], axis = 1).to_numpy()
y_test_latent =  df_x_test_srs[['SWAN_IN_Avg' ,'SWAN_HY_Avg' ,'SRS_RRB' , 'SRS_SCI']]


In [45]:
res_train.reshape(-1, 1)

array([[-1.65692084e-02],
       [ 9.24082734e-01],
       [ 1.66483970e+00],
       [-1.65198327e+00],
       [ 7.95931303e-01],
       [-2.32669131e+00],
       [ 1.51089662e+00],
       [-2.61371259e+00],
       [-3.87675082e-01],
       [ 8.83491737e-01],
       [-1.74009458e+00],
       [ 5.76524501e+00],
       [ 2.06210094e+00],
       [-1.19359824e+00],
       [-1.29981548e+00],
       [-6.68354716e-01],
       [-1.97433220e+00],
       [-1.23283675e+00],
       [ 1.24497486e+00],
       [-2.40678477e+00],
       [ 5.56050279e-01],
       [-7.19890456e-01],
       [-1.19246400e+00],
       [-3.70494784e+00],
       [-2.07084337e+00],
       [-2.66495136e+00],
       [-1.78917321e+00],
       [-5.94244696e-01],
       [-2.76157353e+00],
       [-1.33323172e+00],
       [-1.02176795e+00],
       [-7.75621411e-01],
       [ 2.33651865e+00],
       [-2.40757409e+00],
       [ 4.86846673e+00],
       [-1.14052835e-01],
       [-1.89800959e+00],
       [ 2.01365790e+00],
       [ 1.5

In [46]:
x_train_latent_r = np.hstack((x_train_latent[100:], res_train))
x_test_latent_r = np.hstack((x_test_latent, res_test))

In [47]:
print(x_train_latent)

[[ 1.00000000e+00  7.62320300e+00 -2.80102491e-01 ...  5.32672703e-01
  -6.11483634e-01  4.91583735e-01]
 [ 0.00000000e+00  7.98745100e+00  7.65302896e-01 ...  8.33141685e-01
   9.18909550e-01  1.64636776e-01]
 [ 1.00000000e+00  8.09685100e+00  1.07074156e-02 ... -3.33051026e-01
  -5.90445846e-03 -2.43863016e-01]
 ...
 [ 0.00000000e+00  1.61194380e+01  2.12396562e-01 ... -1.79219812e-01
  -1.13746904e-01 -2.43316665e-01]
 [ 1.00000000e+00  7.56308400e+00 -4.94379997e-02 ... -1.65504023e-01
   1.33824021e-01 -2.26459935e-01]
 [ 0.00000000e+00  1.53578590e+01 -7.80092776e-02 ...  4.30271745e-01
   7.35621870e-01 -2.36293152e-01]]


In [48]:
y_t_l_scale , y_l_m , y_l_s = normalize(y_train_latent)
y_t_l_scale

,SWAN_IN_Avg,SWAN_HY_Avg,SRS_RRB,SRS_SCI
0,-0.672042,1.641460,0.621176,1.040625
1,0.555704,0.973792,1.274470,0.455723
2,-0.010948,-1.029209,0.359859,0.494716
3,-0.483158,0.783030,0.751835,0.884651
4,-0.766484,-0.552304,0.359859,0.182768
...,...,...,...,...
711,0.839030,0.973792,-0.293435,-1.259991
712,0.461262,0.115363,1.535788,1.430560
713,-2.183114,-2.841450,-1.208046,-1.532946
714,-1.144252,0.878411,-0.032117,0.143775


In [167]:
checkpointer = ModelCheckpoint(filepath='Best_model_latent.h5', monitor='val_loss', verbose=1, save_best_only=True)
model = get_model(x_train_latent.shape[1] , y_t_l_scale.shape[1])
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='mse', verbose=1, save_best_only=True, mode='max')

###
early = EarlyStopping(monitor="mse", mode="max", patience=10, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_loss", mode="min", patience=150, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
###
hist = model.fit(
    x_train_latent, 
    y_t_l_scale.to_numpy(), 
    validation_split = 0.1,
    batch_size=300, epochs=800, verbose=2, shuffle=True, 
    callbacks=[checkpointer, redonplat])

model.load_weights('Best_model_latent.h5')

Train on 644 samples, validate on 72 samples
Epoch 1/800
 - 1s - loss: 2.7959 - val_loss: 2.0968

Epoch 00001: val_loss improved from inf to 2.09676, saving model to Best_model_latent.h5
Epoch 2/800
 - 0s - loss: 1.9374 - val_loss: 1.4267

Epoch 00002: val_loss improved from 2.09676 to 1.42673, saving model to Best_model_latent.h5
Epoch 3/800
 - 0s - loss: 1.3914 - val_loss: 1.0335

Epoch 00003: val_loss improved from 1.42673 to 1.03352, saving model to Best_model_latent.h5
Epoch 4/800
 - 0s - loss: 1.1110 - val_loss: 0.8905

Epoch 00004: val_loss improved from 1.03352 to 0.89049, saving model to Best_model_latent.h5
Epoch 5/800
 - 0s - loss: 1.0494 - val_loss: 0.9107

Epoch 00005: val_loss did not improve from 0.89049
Epoch 6/800
 - 0s - loss: 1.1042 - val_loss: 0.9590

Epoch 00006: val_loss did not improve from 0.89049
Epoch 7/800
 - 0s - loss: 1.1404 - val_loss: 0.9608

Epoch 00007: val_loss did not improve from 0.89049
Epoch 8/800
 - 0s - loss: 1.1218 - val_loss: 0.9289

Epoch 0000


Epoch 00072: val_loss did not improve from 0.78539
Epoch 73/800
 - 0s - loss: 0.9299 - val_loss: 0.8077

Epoch 00073: val_loss did not improve from 0.78539
Epoch 74/800
 - 0s - loss: 0.9268 - val_loss: 0.7885

Epoch 00074: val_loss did not improve from 0.78539
Epoch 75/800
 - 0s - loss: 0.9302 - val_loss: 0.7807

Epoch 00075: val_loss improved from 0.78539 to 0.78067, saving model to Best_model_latent.h5
Epoch 76/800
 - 0s - loss: 0.9323 - val_loss: 0.7873

Epoch 00076: val_loss did not improve from 0.78067
Epoch 77/800
 - 0s - loss: 0.9284 - val_loss: 0.8018

Epoch 00077: val_loss did not improve from 0.78067
Epoch 78/800
 - 0s - loss: 0.9255 - val_loss: 0.8128

Epoch 00078: val_loss did not improve from 0.78067
Epoch 79/800
 - 0s - loss: 0.9242 - val_loss: 0.8252

Epoch 00079: val_loss did not improve from 0.78067
Epoch 80/800
 - 0s - loss: 0.9256 - val_loss: 0.8301

Epoch 00080: val_loss did not improve from 0.78067
Epoch 81/800
 - 0s - loss: 0.9270 - val_loss: 0.8158

Epoch 00081:

 - 0s - loss: 0.8864 - val_loss: 0.8370

Epoch 00150: val_loss did not improve from 0.78067
Epoch 151/800
 - 0s - loss: 0.8846 - val_loss: 0.8257

Epoch 00151: val_loss did not improve from 0.78067
Epoch 152/800
 - 0s - loss: 0.8871 - val_loss: 0.8126

Epoch 00152: val_loss did not improve from 0.78067
Epoch 153/800
 - 0s - loss: 0.8856 - val_loss: 0.8175

Epoch 00153: val_loss did not improve from 0.78067
Epoch 154/800
 - 0s - loss: 0.8840 - val_loss: 0.8397

Epoch 00154: val_loss did not improve from 0.78067
Epoch 155/800
 - 0s - loss: 0.8885 - val_loss: 0.8470

Epoch 00155: val_loss did not improve from 0.78067
Epoch 156/800
 - 0s - loss: 0.8865 - val_loss: 0.8386

Epoch 00156: val_loss did not improve from 0.78067
Epoch 157/800
 - 0s - loss: 0.8833 - val_loss: 0.8246

Epoch 00157: val_loss did not improve from 0.78067
Epoch 158/800
 - 0s - loss: 0.8808 - val_loss: 0.8260

Epoch 00158: val_loss did not improve from 0.78067
Epoch 159/800
 - 0s - loss: 0.8805 - val_loss: 0.8326

Epoch


Epoch 00227: val_loss did not improve from 0.78067
Epoch 228/800
 - 0s - loss: 0.8382 - val_loss: 0.8665

Epoch 00228: val_loss did not improve from 0.78067
Epoch 229/800
 - 0s - loss: 0.8372 - val_loss: 0.8670

Epoch 00229: val_loss did not improve from 0.78067
Epoch 230/800
 - 0s - loss: 0.8365 - val_loss: 0.8672

Epoch 00230: val_loss did not improve from 0.78067
Epoch 231/800
 - 0s - loss: 0.8355 - val_loss: 0.8663

Epoch 00231: val_loss did not improve from 0.78067
Epoch 232/800
 - 0s - loss: 0.8350 - val_loss: 0.8670

Epoch 00232: val_loss did not improve from 0.78067
Epoch 233/800
 - 0s - loss: 0.8344 - val_loss: 0.8696

Epoch 00233: val_loss did not improve from 0.78067
Epoch 234/800
 - 0s - loss: 0.8338 - val_loss: 0.8720

Epoch 00234: val_loss did not improve from 0.78067
Epoch 235/800
 - 0s - loss: 0.8335 - val_loss: 0.8750

Epoch 00235: val_loss did not improve from 0.78067
Epoch 236/800
 - 0s - loss: 0.8334 - val_loss: 0.8774

Epoch 00236: val_loss did not improve from 0.

 - 0s - loss: 0.8264 - val_loss: 0.8870

Epoch 00305: val_loss did not improve from 0.78067
Epoch 306/800
 - 0s - loss: 0.8265 - val_loss: 0.8876

Epoch 00306: val_loss did not improve from 0.78067
Epoch 307/800
 - 0s - loss: 0.8263 - val_loss: 0.8897

Epoch 00307: val_loss did not improve from 0.78067
Epoch 308/800
 - 0s - loss: 0.8261 - val_loss: 0.8915

Epoch 00308: val_loss did not improve from 0.78067
Epoch 309/800
 - 0s - loss: 0.8261 - val_loss: 0.8916

Epoch 00309: val_loss did not improve from 0.78067
Epoch 310/800
 - 0s - loss: 0.8260 - val_loss: 0.8903

Epoch 00310: val_loss did not improve from 0.78067
Epoch 311/800
 - 0s - loss: 0.8258 - val_loss: 0.8887

Epoch 00311: val_loss did not improve from 0.78067
Epoch 312/800
 - 0s - loss: 0.8258 - val_loss: 0.8872

Epoch 00312: val_loss did not improve from 0.78067
Epoch 313/800
 - 0s - loss: 0.8257 - val_loss: 0.8862

Epoch 00313: val_loss did not improve from 0.78067
Epoch 314/800
 - 0s - loss: 0.8256 - val_loss: 0.8865

Epoch

...


Epoch 00459: val_loss did not improve from 0.78067
Epoch 460/800
 - 0s - loss: 0.8196 - val_loss: 0.8889

Epoch 00460: val_loss did not improve from 0.78067
Epoch 461/800
 - 0s - loss: 0.8196 - val_loss: 0.8886

Epoch 00461: val_loss did not improve from 0.78067
Epoch 462/800
 - 0s - loss: 0.8196 - val_loss: 0.8884

Epoch 00462: val_loss did not improve from 0.78067
Epoch 463/800
 - 0s - loss: 0.8196 - val_loss: 0.8882

Epoch 00463: val_loss did not improve from 0.78067
Epoch 464/800
 - 0s - loss: 0.8196 - val_loss: 0.8882

Epoch 00464: val_loss did not improve from 0.78067
Epoch 465/800
 - 0s - loss: 0.8196 - val_loss: 0.8883

Epoch 00465: val_loss did not improve from 0.78067
Epoch 466/800
 - 0s - loss: 0.8196 - val_loss: 0.8885

Epoch 00466: val_loss did not improve from 0.78067
Epoch 467/800
 - 0s - loss: 0.8196 - val_loss: 0.8887

Epoch 00467: val_loss did not improve from 0.78067
Epoch 468/800
 - 0s - loss: 0.8196 - val_loss: 0.8890

Epoch 00468: val_loss did not improve from 0.

 - 0s - loss: 0.8191 - val_loss: 0.8896

Epoch 00536: val_loss did not improve from 0.78067
Epoch 537/800
 - 0s - loss: 0.8191 - val_loss: 0.8896

Epoch 00537: val_loss did not improve from 0.78067
Epoch 538/800
 - 0s - loss: 0.8191 - val_loss: 0.8896

Epoch 00538: val_loss did not improve from 0.78067
Epoch 539/800
 - 0s - loss: 0.8191 - val_loss: 0.8896

Epoch 00539: val_loss did not improve from 0.78067
Epoch 540/800
 - 0s - loss: 0.8191 - val_loss: 0.8896

Epoch 00540: val_loss did not improve from 0.78067
Epoch 541/800
 - 0s - loss: 0.8191 - val_loss: 0.8896

Epoch 00541: val_loss did not improve from 0.78067
Epoch 542/800
 - 0s - loss: 0.8191 - val_loss: 0.8895

Epoch 00542: val_loss did not improve from 0.78067
Epoch 543/800
 - 0s - loss: 0.8191 - val_loss: 0.8895

Epoch 00543: val_loss did not improve from 0.78067
Epoch 544/800
 - 0s - loss: 0.8191 - val_loss: 0.8894

Epoch 00544: val_loss did not improve from 0.78067
Epoch 545/800
 - 0s - loss: 0.8191 - val_loss: 0.8894

Epoch


Epoch 00613: val_loss did not improve from 0.78067
Epoch 614/800
 - 0s - loss: 0.8190 - val_loss: 0.8891

Epoch 00614: val_loss did not improve from 0.78067
Epoch 615/800
 - 0s - loss: 0.8190 - val_loss: 0.8891

Epoch 00615: val_loss did not improve from 0.78067
Epoch 616/800
 - 0s - loss: 0.8190 - val_loss: 0.8891

Epoch 00616: val_loss did not improve from 0.78067
Epoch 617/800
 - 0s - loss: 0.8190 - val_loss: 0.8892

Epoch 00617: val_loss did not improve from 0.78067
Epoch 618/800
 - 0s - loss: 0.8190 - val_loss: 0.8892

Epoch 00618: val_loss did not improve from 0.78067
Epoch 619/800
 - 0s - loss: 0.8190 - val_loss: 0.8892

Epoch 00619: val_loss did not improve from 0.78067
Epoch 620/800
 - 0s - loss: 0.8190 - val_loss: 0.8892

Epoch 00620: val_loss did not improve from 0.78067
Epoch 621/800
 - 0s - loss: 0.8190 - val_loss: 0.8892

Epoch 00621: val_loss did not improve from 0.78067
Epoch 622/800
 - 0s - loss: 0.8190 - val_loss: 0.8892

Epoch 00622: val_loss did not improve from 0.


Epoch 00690: val_loss did not improve from 0.78067
Epoch 691/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00691: val_loss did not improve from 0.78067
Epoch 692/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00692: val_loss did not improve from 0.78067
Epoch 693/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00693: val_loss did not improve from 0.78067
Epoch 694/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00694: val_loss did not improve from 0.78067
Epoch 695/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00695: val_loss did not improve from 0.78067
Epoch 696/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00696: val_loss did not improve from 0.78067
Epoch 697/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00697: val_loss did not improve from 0.78067
Epoch 698/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00698: val_loss did not improve from 0.78067
Epoch 699/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00699: val_loss did not improve from 0.

 - 0s - loss: 0.8190 - val_loss: 0.8894

Epoch 00768: val_loss did not improve from 0.78067
Epoch 769/800
 - 0s - loss: 0.8190 - val_loss: 0.8894

Epoch 00769: val_loss did not improve from 0.78067
Epoch 770/800
 - 0s - loss: 0.8190 - val_loss: 0.8894

Epoch 00770: val_loss did not improve from 0.78067
Epoch 771/800
 - 0s - loss: 0.8190 - val_loss: 0.8894

Epoch 00771: val_loss did not improve from 0.78067
Epoch 772/800
 - 0s - loss: 0.8190 - val_loss: 0.8894

Epoch 00772: val_loss did not improve from 0.78067
Epoch 773/800
 - 0s - loss: 0.8190 - val_loss: 0.8894

Epoch 00773: val_loss did not improve from 0.78067
Epoch 774/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00774: val_loss did not improve from 0.78067
Epoch 775/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00775: val_loss did not improve from 0.78067
Epoch 776/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch 00776: val_loss did not improve from 0.78067
Epoch 777/800
 - 0s - loss: 0.8190 - val_loss: 0.8895

Epoch

In [168]:
y_pred_l = model.predict(x_test_latent)
y_test_latent_scale = normalize_p(y_test_latent , y_l_m , y_l_s)

In [169]:
for i in range(len(y_test_latent.columns)):
    print(y_test_latent.columns[i] , ':', mean_squared_error( y_test_latent_scale.to_numpy()[:,i] , y_pred_l[:,i]) )

SWAN_IN_Avg : 1.0985934700264264
SWAN_HY_Avg : 0.9885638346384623
SRS_RRB : 0.838119536948195
SRS_SCI : 0.7500708456313906


## Mean Baseline For latent 

In [53]:
y_latent_mean = np.zeros(y_test_latent.shape) + np.mean(y_t_l_scale , axis=0).to_numpy()
for i in range(len(y_test_latent.columns)):
    print(y_test_latent.columns[i] , ':', mean_squared_error( y_test_latent_scale.to_numpy()[:,i] , y_latent_mean[:,i]) )

SWAN_IN_Avg : 1.1118545009115888
SWAN_HY_Avg : 1.0224701184257916
SRS_RRB : 0.875080479818877
SRS_SCI : 0.776254368094447


# With Residuals

In [137]:
y_t_l_scale_r = y_t_l_scale[100:]

In [150]:
checkpointer = ModelCheckpoint(filepath='Best_model_latent.h5', monitor='val_loss', verbose=1, save_best_only=True)
model = get_model(x_train_latent_r.shape[1] , y_t_l_scale_r.shape[1])
file_path = "best_model.h5"
checkpoint = ModelCheckpoint(file_path, monitor='mse', verbose=1, save_best_only=True, mode='max')

###
early = EarlyStopping(monitor="mse", mode="max", patience=10, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_loss", mode="min", patience=150, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early
###
hist = model.fit(
    x_train_latent_r, 
    y_t_l_scale_r.to_numpy(), 
    validation_split = 0.1,
    batch_size=300, epochs=800, verbose=2, shuffle=True, 
    callbacks=[checkpointer, redonplat])

model.load_weights('Best_model_latent.h5')

Train on 554 samples, validate on 62 samples
Epoch 1/800
 - 1s - loss: 3.0392 - val_loss: 3.4055

Epoch 00001: val_loss improved from inf to 3.40547, saving model to Best_model_latent.h5
Epoch 2/800
 - 0s - loss: 2.3650 - val_loss: 2.6048

Epoch 00002: val_loss improved from 3.40547 to 2.60478, saving model to Best_model_latent.h5
Epoch 3/800
 - 0s - loss: 1.8623 - val_loss: 1.9972

Epoch 00003: val_loss improved from 2.60478 to 1.99718, saving model to Best_model_latent.h5
Epoch 4/800
 - 0s - loss: 1.4869 - val_loss: 1.5493

Epoch 00004: val_loss improved from 1.99718 to 1.54925, saving model to Best_model_latent.h5
Epoch 5/800
 - 0s - loss: 1.2421 - val_loss: 1.2409

Epoch 00005: val_loss improved from 1.54925 to 1.24091, saving model to Best_model_latent.h5
Epoch 6/800
 - 0s - loss: 1.1043 - val_loss: 1.0487

Epoch 00006: val_loss improved from 1.24091 to 1.04866, saving model to Best_model_latent.h5
Epoch 7/800
 - 0s - loss: 1.0388 - val_loss: 0.9496

Epoch 00007: val_loss improved


Epoch 00067: val_loss did not improve from 0.82579
Epoch 68/800
 - 0s - loss: 0.9037 - val_loss: 0.8284

Epoch 00068: val_loss did not improve from 0.82579
Epoch 69/800
 - 0s - loss: 0.9027 - val_loss: 0.8280

Epoch 00069: val_loss did not improve from 0.82579
Epoch 70/800
 - 0s - loss: 0.9016 - val_loss: 0.8271

Epoch 00070: val_loss did not improve from 0.82579
Epoch 71/800
 - 0s - loss: 0.9007 - val_loss: 0.8280

Epoch 00071: val_loss did not improve from 0.82579
Epoch 72/800
 - 0s - loss: 0.8998 - val_loss: 0.8280

Epoch 00072: val_loss did not improve from 0.82579
Epoch 73/800
 - 0s - loss: 0.8989 - val_loss: 0.8281

Epoch 00073: val_loss did not improve from 0.82579
Epoch 74/800
 - 0s - loss: 0.8984 - val_loss: 0.8282

Epoch 00074: val_loss did not improve from 0.82579
Epoch 75/800
 - 0s - loss: 0.8971 - val_loss: 0.8265

Epoch 00075: val_loss did not improve from 0.82579
Epoch 76/800
 - 0s - loss: 0.8961 - val_loss: 0.8257

Epoch 00076: val_loss improved from 0.82579 to 0.82570


Epoch 00144: val_loss did not improve from 0.82507
Epoch 145/800
 - 0s - loss: 0.8427 - val_loss: 0.8404

Epoch 00145: val_loss did not improve from 0.82507
Epoch 146/800
 - 0s - loss: 0.8417 - val_loss: 0.8396

Epoch 00146: val_loss did not improve from 0.82507
Epoch 147/800
 - 0s - loss: 0.8407 - val_loss: 0.8401

Epoch 00147: val_loss did not improve from 0.82507
Epoch 148/800
 - 0s - loss: 0.8400 - val_loss: 0.8403

Epoch 00148: val_loss did not improve from 0.82507
Epoch 149/800
 - 0s - loss: 0.8393 - val_loss: 0.8397

Epoch 00149: val_loss did not improve from 0.82507
Epoch 150/800
 - 0s - loss: 0.8389 - val_loss: 0.8395

Epoch 00150: val_loss did not improve from 0.82507
Epoch 151/800
 - 0s - loss: 0.8377 - val_loss: 0.8431

Epoch 00151: val_loss did not improve from 0.82507
Epoch 152/800
 - 0s - loss: 0.8373 - val_loss: 0.8448

Epoch 00152: val_loss did not improve from 0.82507
Epoch 153/800
 - 0s - loss: 0.8367 - val_loss: 0.8403

Epoch 00153: val_loss did not improve from 0.


Epoch 00222: val_loss did not improve from 0.82507
Epoch 223/800
 - 0s - loss: 0.7780 - val_loss: 0.8512

Epoch 00223: val_loss did not improve from 0.82507
Epoch 224/800
 - 0s - loss: 0.7767 - val_loss: 0.8623

Epoch 00224: val_loss did not improve from 0.82507
Epoch 225/800
 - 0s - loss: 0.7753 - val_loss: 0.8637

Epoch 00225: val_loss did not improve from 0.82507
Epoch 226/800
 - 0s - loss: 0.7750 - val_loss: 0.8600

Epoch 00226: val_loss did not improve from 0.82507
Epoch 227/800
 - 0s - loss: 0.7737 - val_loss: 0.8595

Epoch 00227: val_loss did not improve from 0.82507
Epoch 228/800
 - 0s - loss: 0.7725 - val_loss: 0.8587

Epoch 00228: val_loss did not improve from 0.82507
Epoch 229/800
 - 0s - loss: 0.7720 - val_loss: 0.8633

Epoch 00229: val_loss did not improve from 0.82507
Epoch 230/800
 - 0s - loss: 0.7710 - val_loss: 0.8645

Epoch 00230: val_loss did not improve from 0.82507
Epoch 231/800
 - 0s - loss: 0.7704 - val_loss: 0.8639

Epoch 00231: val_loss did not improve from 0.

 - 0s - loss: 0.7631 - val_loss: 0.8642

Epoch 00299: val_loss did not improve from 0.82507
Epoch 300/800
 - 0s - loss: 0.7630 - val_loss: 0.8645

Epoch 00300: val_loss did not improve from 0.82507
Epoch 301/800
 - 0s - loss: 0.7629 - val_loss: 0.8646

Epoch 00301: val_loss did not improve from 0.82507
Epoch 302/800
 - 0s - loss: 0.7629 - val_loss: 0.8651

Epoch 00302: val_loss did not improve from 0.82507
Epoch 303/800
 - 0s - loss: 0.7628 - val_loss: 0.8655

Epoch 00303: val_loss did not improve from 0.82507
Epoch 304/800
 - 0s - loss: 0.7627 - val_loss: 0.8651

Epoch 00304: val_loss did not improve from 0.82507
Epoch 305/800
 - 0s - loss: 0.7627 - val_loss: 0.8647

Epoch 00305: val_loss did not improve from 0.82507
Epoch 306/800
 - 0s - loss: 0.7626 - val_loss: 0.8650

Epoch 00306: val_loss did not improve from 0.82507
Epoch 307/800
 - 0s - loss: 0.7625 - val_loss: 0.8649

Epoch 00307: val_loss did not improve from 0.82507
Epoch 308/800
 - 0s - loss: 0.7624 - val_loss: 0.8647

Epoch

...

 - 0s - loss: 0.7563 - val_loss: 0.8679

Epoch 00453: val_loss did not improve from 0.82507
Epoch 454/800
 - 0s - loss: 0.7563 - val_loss: 0.8680

Epoch 00454: val_loss did not improve from 0.82507
Epoch 455/800
 - 0s - loss: 0.7563 - val_loss: 0.8679

Epoch 00455: val_loss did not improve from 0.82507
Epoch 456/800
 - 0s - loss: 0.7563 - val_loss: 0.8679

Epoch 00456: val_loss did not improve from 0.82507
Epoch 457/800
 - 0s - loss: 0.7563 - val_loss: 0.8679

Epoch 00457: val_loss did not improve from 0.82507
Epoch 458/800
 - 0s - loss: 0.7563 - val_loss: 0.8678

Epoch 00458: val_loss did not improve from 0.82507
Epoch 459/800
 - 0s - loss: 0.7563 - val_loss: 0.8678

Epoch 00459: val_loss did not improve from 0.82507
Epoch 460/800
 - 0s - loss: 0.7563 - val_loss: 0.8679

Epoch 00460: val_loss did not improve from 0.82507
Epoch 461/800
 - 0s - loss: 0.7562 - val_loss: 0.8679

Epoch 00461: val_loss did not improve from 0.82507
Epoch 462/800
 - 0s - loss: 0.7562 - val_loss: 0.8679

Epoch


Epoch 00530: val_loss did not improve from 0.82507
Epoch 531/800
 - 0s - loss: 0.7557 - val_loss: 0.8685

Epoch 00531: val_loss did not improve from 0.82507
Epoch 532/800
 - 0s - loss: 0.7557 - val_loss: 0.8685

Epoch 00532: val_loss did not improve from 0.82507

Epoch 00532: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
Epoch 533/800
 - 0s - loss: 0.7557 - val_loss: 0.8685

Epoch 00533: val_loss did not improve from 0.82507
Epoch 534/800
 - 0s - loss: 0.7557 - val_loss: 0.8685

Epoch 00534: val_loss did not improve from 0.82507
Epoch 535/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00535: val_loss did not improve from 0.82507
Epoch 536/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00536: val_loss did not improve from 0.82507
Epoch 537/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00537: val_loss did not improve from 0.82507
Epoch 538/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00538: val_loss did not improve from 0.82507
Epoch 539/800
 - 0

 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00607: val_loss did not improve from 0.82507
Epoch 608/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00608: val_loss did not improve from 0.82507
Epoch 609/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00609: val_loss did not improve from 0.82507
Epoch 610/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00610: val_loss did not improve from 0.82507
Epoch 611/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00611: val_loss did not improve from 0.82507
Epoch 612/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00612: val_loss did not improve from 0.82507
Epoch 613/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00613: val_loss did not improve from 0.82507
Epoch 614/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00614: val_loss did not improve from 0.82507
Epoch 615/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch 00615: val_loss did not improve from 0.82507
Epoch 616/800
 - 0s - loss: 0.7557 - val_loss: 0.8684

Epoch

 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00684: val_loss did not improve from 0.82507
Epoch 685/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00685: val_loss did not improve from 0.82507
Epoch 686/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00686: val_loss did not improve from 0.82507
Epoch 687/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00687: val_loss did not improve from 0.82507
Epoch 688/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00688: val_loss did not improve from 0.82507
Epoch 689/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00689: val_loss did not improve from 0.82507
Epoch 690/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00690: val_loss did not improve from 0.82507
Epoch 691/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00691: val_loss did not improve from 0.82507
Epoch 692/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00692: val_loss did not improve from 0.82507
Epoch 693/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch


Epoch 00761: val_loss did not improve from 0.82507
Epoch 762/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00762: val_loss did not improve from 0.82507
Epoch 763/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00763: val_loss did not improve from 0.82507
Epoch 764/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00764: val_loss did not improve from 0.82507
Epoch 765/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00765: val_loss did not improve from 0.82507
Epoch 766/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00766: val_loss did not improve from 0.82507
Epoch 767/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00767: val_loss did not improve from 0.82507
Epoch 768/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00768: val_loss did not improve from 0.82507
Epoch 769/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00769: val_loss did not improve from 0.82507
Epoch 770/800
 - 0s - loss: 0.7556 - val_loss: 0.8683

Epoch 00770: val_loss did not improve from 0.

In [151]:
y_pred_l_r = model.predict(x_test_latent_r)

In [152]:
#y_test_latent_scale = normalize_p(y_test_latent , y_l_m , y_l_s)

In [153]:
for i in range(len(y_test_latent.columns)):
    print(y_test_latent.columns[i] , ':', mean_squared_error( y_test_latent_scale.to_numpy()[:,i] , y_pred_l_r[:,i]) )

SWAN_IN_Avg : 1.0898022979903794
SWAN_HY_Avg : 1.0334526228074459
SRS_RRB : 0.8606484071675834
SRS_SCI : 0.7695837318013414


# SVM

In [62]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor


In [63]:
def fit_xgboost_with_grid_search(X_train, y_train):

    # Create gradient boosting object
    xgbr = XGBRegressor(verbosity=1, max_depth=10, reg_lambda=1) 
    
    # Grid search 
    parameters = {'max_depth':[1,2,4], 'reg_lambda':[4, 8, 16,  20], 'min_child_weight':[6,  12, 25, 30]}
    clf = GridSearchCV(xgbr, parameters, scoring='neg_mean_squared_error', n_jobs=4, iid=False, cv=5)
    clf.fit(X_train, y_train)
    
    print("Best parameters for XGBoost: " + str(clf.best_params_))
            
    return clf

In [73]:
#rdf = fit_random_forest_with_grid_search(x_train_latent, y_t_l_scale.to_numpy())

In [ ]:
rdf.best_score_

In [ ]:
y_pred = rdf.predict(x_test_latent)

In [ ]:
for i in range(len(y_test_latent.columns)):
    print(y_test_latent.columns[i] , ':', mean_squared_error( y_test_latent_scale.to_numpy()[:,i] , y_pred[:,i]) )